## Belgian Traffic Signs Classification 

The dataset of BelgianTS is wellknown and probably it has various versions with different sizes. Here I used the dataset that contains 62 classes of images with 4575 images for training and 2520 for validation.

Some of simplified models have been used step by step to check the validation set accuracy. A simple 2 layers feed-forward NN model shows 93.89% accuracy, while an addition of one Conv2D+MaxPooling2D gives 95.3% accuracy.

Two pairs of Conv2D+MaxPooling2D alon with two Dense layers helps to achieve 97% validation accuracy. At the end we see that the accuracy increases to 97.8% when we add Dropout layer after each MaxPooling.

One observation for this dataset is: grey scale images results in faster training, whereas RGB images give higher accuracy.


In [1]:
import os
from skimage import data
#from skimage.viewer import ImageViewer
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

### Below three cells are related to ETL process of data preprocessing 

In [2]:
def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                   if os.path.isdir(os.path.join(data_directory, d))]
    labels = []
    images = []
    for d in directories:
        label_directory = os.path.join(data_directory, d)
        file_names = [os.path.join(label_directory, f) 
                      for f in os.listdir(label_directory) 
                      if f.endswith(".ppm")]
        for f in file_names:
            images.append(data.imread(f)) #skimage theke data import kore imread() call kora hoise 
            labels.append(int(d))
    return images, labels

In [3]:
cwd = os.getcwd()
train_data_directory = os.path.join(cwd, "Training")
test_data_directory = os.path.join(cwd, "Testing")

x_train, y_train = load_data(train_data_directory)
x_test, y_test = load_data(test_data_directory)

#callbacks = myCallback()

In [4]:
from skimage import transform
from skimage.color import rgb2gray

Trainimages = [transform.resize(image, (28, 28)) for image in x_train]
Trainimages = np.array(Trainimages)
#Trainimages = rgb2gray(Trainimages)

Testimages = [transform.resize(image, (28, 28)) for image in x_test]
Testimages = np.array(Testimages)
#Testimages = rgb2gray(Testimages)

### The siplest model with two dense layers, Validation accurcy = 93.89%

In [53]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy')>0.935):
      print("\nReached 93.5% accuracy on test set, so cancelling training!")
      self.model.stop_training = True

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28,3)),
  tf.keras.layers.Dense(256, activation=tf.nn.relu),
  tf.keras.layers.Dense(62, activation=tf.nn.softmax)

])

callbacks = myCallback()

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(np.array(Trainimages), np.array(y_train), validation_data=(Testimages, np.array(y_test)), 
          epochs=100, callbacks=[callbacks])

Train on 4575 samples, validate on 2520 samples
Epoch 1/100
4575/4575 [==============================] - 3s 648us/sample - loss: 2.1950 - accuracy: 0.5128 - val_loss: 1.4593 - val_accuracy: 0.6762
Epoch 2/100
4575/4575 [==============================] - 2s 518us/sample - loss: 1.0671 - accuracy: 0.7534 - val_loss: 0.9799 - val_accuracy: 0.7762
Epoch 3/100
4575/4575 [==============================] - 2s 491us/sample - loss: 0.7245 - accuracy: 0.8345 - val_loss: 0.8533 - val_accuracy: 0.7639
Epoch 4/100
4575/4575 [==============================] - 2s 498us/sample - loss: 0.5326 - accuracy: 0.8748 - val_loss: 0.6038 - val_accuracy: 0.8385
Epoch 5/100
4575/4575 [==============================] - 2s 481us/sample - loss: 0.4025 - accuracy: 0.9104 - val_loss: 0.5286 - val_accuracy: 0.8508
Epoch 6/100
4575/4575 [==============================] - 2s 430us/sample - loss: 0.3227 - accuracy: 0.9329 - val_loss: 0.5336 - val_accuracy: 0.8381
Epoch 7/100
4575/4575 [==============================] - 2

### Model with one pair of Conv2D+MaxPooling2D  along with two Dense layers, Validation accurcy = 95.3%



In [55]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy')>=0.95):
      print("\nReached 96% accuracy on test set, so cancelling training!")
      self.model.stop_training = True

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, input_shape=(28, 28, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(62, activation=tf.nn.softmax)

])

callbacks = myCallback()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

model.fit(np.array(Trainimages), np.array(y_train), validation_data=(Testimages, np.array(y_test)), 
          epochs=100, callbacks=[callbacks])

#model.evaluate(np.array(Testimages), np.array(y_test))

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 26, 26, 64)        1792      
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 13, 13, 64)        0         
_________________________________________________________________
flatten_29 (Flatten)         (None, 10816)             0         
_________________________________________________________________
dense_58 (Dense)             (None, 256)               2769152   
_________________________________________________________________
dense_59 (Dense)             (None, 62)                15934     
Total params: 2,786,878
Trainable params: 2,786,878
Non-trainable params: 0
_________________________________________________________________
Train on 4575 samples, validate on 2520 samples
Epoch 1/100
4575/4575 [==============================] - 15s 

### Model with two pair of Conv2D+MaxPooling2D along with two Dense layers, Validation accurcy = 97%

In [58]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy')>0.97):
      print("\nReached 97% accuracy on test set, so cancelling training!")
      self.model.stop_training = True
        
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, input_shape=(28, 28, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(62, activation=tf.nn.softmax)

])

callbacks = myCallback()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

model.fit(Trainimages, np.array(y_train),validation_data=(Testimages, np.array(y_test)),  epochs=100, callbacks=[callbacks])

#model.evaluate(Testimages, np.array(y_test))

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_50 (Conv2D)           (None, 26, 26, 64)        1792      
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_32 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dense_64 (Dense)             (None, 256)               409856    
_________________________________________________________________
dense_65 (Dense)             (None, 62)              

## Model with two sets of Conv2D+MaxPooling2D+Dropout layers along with two Dense layers, Validation accurcy = 97.8%

In [33]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy')>=0.9775):
      print("\nReached 97.75% accuracy on test set, so cancelling training!")
      self.model.stop_training = True

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3),  activation=tf.nn.relu, input_shape=(28, 28, 3)), #padding='SAME', strides=(2, 2),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.7),
    tf.keras.layers.Conv2D(64, (3,3),  activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.7),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(62, activation=tf.nn.softmax)

])

callbacks = myCallback()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

model.fit(Trainimages, np.array(y_train),validation_data=(Testimages, np.array(y_test)),  epochs=100, callbacks=[callbacks])

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_55 (Conv2D)           (None, 26, 26, 64)        1792      
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 13, 13, 64)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_54 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_27 (Flatten)         (None, 1600)            

4575/4575 [==============================] - 12s 3ms/sample - loss: 0.0878 - accuracy: 0.9727 - val_loss: 0.1452 - val_accuracy: 0.9651
Epoch 41/100
4575/4575 [==============================] - 12s 3ms/sample - loss: 0.0823 - accuracy: 0.9711 - val_loss: 0.1692 - val_accuracy: 0.9615
Epoch 42/100
4575/4575 [==============================] - 12s 3ms/sample - loss: 0.0840 - accuracy: 0.9722 - val_loss: 0.1356 - val_accuracy: 0.9690
Epoch 43/100
4575/4575 [==============================] - 12s 3ms/sample - loss: 0.0889 - accuracy: 0.9733 - val_loss: 0.1454 - val_accuracy: 0.9663
Epoch 44/100
4575/4575 [==============================] - 12s 3ms/sample - loss: 0.0875 - accuracy: 0.9729 - val_loss: 0.1437 - val_accuracy: 0.9683
Epoch 45/100
4575/4575 [==============================] - 12s 3ms/sample - loss: 0.0877 - accuracy: 0.9711 - val_loss: 0.1535 - val_accuracy: 0.9663
Epoch 46/100
4575/4575 [==============================] - 12s 3ms/sample - loss: 0.0923 - accuracy: 0.9725 - val_loss: 

4575/4575 [==============================] - 12s 3ms/sample - loss: 0.0708 - accuracy: 0.9803 - val_loss: 0.1766 - val_accuracy: 0.9663
Epoch 96/100
4575/4575 [==============================] - 12s 3ms/sample - loss: 0.0569 - accuracy: 0.9823 - val_loss: 0.1515 - val_accuracy: 0.9726
Epoch 97/100
4575/4575 [==============================] - 12s 3ms/sample - loss: 0.0674 - accuracy: 0.9786 - val_loss: 0.1663 - val_accuracy: 0.9706
Epoch 98/100
4575/4575 [==============================] - 12s 3ms/sample - loss: 0.0616 - accuracy: 0.9808 - val_loss: 0.1700 - val_accuracy: 0.9742
Epoch 99/100
4544/4575 [============================>.] - ETA: 0s - loss: 0.0606 - accuracy: 0.9822
Reached 97.75% accuracy on test set, so cancelling training!
4575/4575 [==============================] - 12s 3ms/sample - loss: 0.0605 - accuracy: 0.9821 - val_loss: 0.1466 - val_accuracy: 0.9782


#### Notes about training
=> 3-3 kernel is superior to others\
\
=> Model with  
Conv2D(256) + MaxPolling2D + Dropout(.5) + \
Conv2D(256) + MaxPolling2D + Dropout(.5) + \
Dense(512) + Dense (62)  \
also gives greater than 97.7% accuracy \
\
=> Padding and Strides do not increase the accuracy, but reduces a little bit

